## import ##

In [ ]:
from scipy.io import readsav
import matplotlib.colors as mcolors
import numpy as np
%matplotlib qt
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pyvista as pv
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import RegularGridInterpolator
import pandas as pd


In [2]:
# 讀取 IDL 的 .sav 檔案
data = readsav("C:/Users/chjan/HMI_output_20231104.sav")

# # 列出所有變數名稱 
# print(data.keys())

# 取出 BP3DZ 資料
BP3DX = data["BP3DX"].T
BP3DY = data["BP3DY"].T
BP3DZ = data["BP3DZ"].T
print("shape:", BP3DX.shape, BP3DY.shape, BP3DZ.shape)

c:\Users\chjan\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\io\_idl.py:339: UserWarning: Using experimental 64-bit array read
  rectypedesc = _read_typedesc(f)


shape: (1687, 1390, 501) (1687, 1390, 501) (1687, 1390, 501)


In [6]:
hmi = pd.read_csv('../read_data/hmi_20231104_cropped_sp.csv')
hmi = hmi.to_numpy()
hmi.shape


(1687, 1390)

In [7]:
# 假設 Bx, By, Bz, X, Y, Z 為已知的磁場向量場
Bx, By, Bz = BP3DX, BP3DY, BP3DZ

print(Bz[154][126])


[-2.0500000e+01 -9.8516865e+00 -6.0753741e+00 -4.1821165e+00
 -3.0286853e+00 -2.2367699e+00 -1.6531098e+00 -1.2053040e+00
 -8.5397446e-01 -5.7489836e-01 -3.5170054e-01 -1.7257340e-01
 -2.8639680e-02  8.6957410e-02  1.7959099e-01  2.5354216e-01
  3.1223729e-01  3.5843903e-01  3.9438751e-01  4.2190909e-01
  4.4249374e-01  4.5736131e-01  4.6751320e-01  4.7377497e-01
  4.7682545e-01  4.7722051e-01  4.7542304e-01  4.7181171e-01
  4.6670184e-01  4.6035349e-01  4.5297956e-01  4.4475952e-01
  4.3583485e-01  4.2632851e-01  4.1634023e-01  4.0595028e-01
  3.9522582e-01  3.8422370e-01  3.7298721e-01  3.6155552e-01
  3.4995672e-01  3.3821744e-01  3.2635581e-01  3.1438994e-01
  3.0233204e-01  2.9019305e-01  2.7798012e-01  2.6570359e-01
  2.5336540e-01  2.4097021e-01  2.2852358e-01  2.1602479e-01
  2.0347813e-01  1.9088551e-01  1.7824727e-01  1.6556473e-01
  1.5283723e-01  1.4006779e-01  1.2725700e-01  1.1440345e-01
  1.0150818e-01  8.8573337e-02  7.5596675e-02  6.2581837e-02
  4.9526211e-02  3.64319

In [33]:
# 定義空間格點 (假設 X, Y, Z 均勻分佈)
Nx, Ny, Nz = Bx.shape
x = np.linspace(0, Nx-1, Nx)  # X 軸範圍為 [0, Nx-1]
y = np.linspace(0, Ny-1, Ny)  # Y 軸範圍為 [0, Ny-1]
z = np.linspace(0, Nz-1, Nz)  # Z 軸範圍為 [0, Nz-1]

def trace_fieldline(start_point, step=10, n_steps=2000):
    """
    使用 Euler 方法追蹤磁場線
    start_point: 初始點 (x, y, z)
    step: 每步移動距離
    n_steps: 最大步數
    """
    traj = [start_point]  # 存儲磁場線上的點
    point = np.array(start_point)
    
    for _ in range(n_steps):

        px, py, pz = np.round(point).astype(int)
        # 取得當前點的磁場值
        B_vector = np.array([
            Bx[px, py, pz],
            By[px, py, pz],
            Bz[px, py, pz]
        ]).flatten()
        # 若磁場過小，停止追蹤
        norm = np.linalg.norm(B_vector)
        if norm < 1e-6:
            print("braek")
            break

        # 歸一化方向，確保方向正確
        B_vector /= np.abs(norm)
        point = point + step * B_vector  # 沿磁場方向前進
        traj.append(point.copy())
        # 確保不超出範圍
        if not (x[0] <= point[0] <= x[-1] and y[0] <= point[1] <= y[-1] and z[0] <= point[2] <= z[-1]):
            break
    return np.array(traj)

# 3D 繪圖
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# 定義幾個磁場線的起點
# x, y 均勻分布在索引範圍內
x_range =  np.round(np.linspace(0, Nx-1, 22)[1:-1])  # 取 10 個點
y_range = np.round(np.linspace(0, Ny-1, 22)[1:-1])  # 取 10 個點

# 生成 (x, y) 網格，z=0
X_grid, Y_grid = np.meshgrid(x_range, y_range)
Z_grid = np.zeros_like(X_grid)  # 全部 z=0

# 組成 start_points
start_points = np.column_stack((X_grid.ravel(), Y_grid.ravel(), Z_grid.ravel()))

for sp in start_points:
    traj = trace_fieldline(sp)
    ax.plot3D(traj[:, 0], traj[:, 1], traj[:, 2], color='r')

# **繪製 HMI 磁場 (Z=0)**
X_hmi, Y_hmi = np.meshgrid(np.arange(Nx), np.arange(Ny), indexing='ij')  # 使用 HMI 原始大小
print(X_hmi.shape,Nx)
ax.plot_surface(X_hmi, Y_hmi, np.zeros_like(hmi),
                cmap='gray', 
                alpha=0.5, vmin=-200, vmax=200,  edgecolors='none')



ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_xlim(x[0], x[-1])
ax.set_ylim(y[0], y[-1])
ax.set_zlim(z[0], z[-1])
ax.set_title('3D streamline')

plt.show()


(1687, 1390) 1687
